TODO:

NER?

Increase the sample size

File: 0704.0001.tex
authorD

intrinsically, there will have lots of missing fields

Better Understand LatexWalker

In [ ]:
!rm -rf Latex-papers

In [1]:
!git clone https://github.com/Luke-Guan-FirstPrinciples/Latex-papers.git

Cloning into 'Latex-papers'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 29 (delta 4), reused 8 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 208.66 KiB | 3.86 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
!pip install bibtexparser pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=b27165a13de01d2e7b73b160b7b69433b3031ec97280498865d0e6aee0468af7
  Stored in directory: /root/.cache/pip/wheels/16/fb/76/306387739cf9d53b1c39b0c8aadbbb17dc05f256756d8fd915
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=e0ab01962955b59ce3747d5bf337bec596e2c20ae7ae789caa01f0e8cfaf3548
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built bibtexparser pylatexenc


In [8]:
import os

In [3]:
!pwd

/content


In [5]:
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode, LatexMacroNode, LatexCharsNode
import re

def clean_latex_text(latex_string):
    walker = LatexWalker(latex_string)
    nodelist, pos, _ = walker.get_latex_nodes(pos=0)
    result = []

    def _extract_text(nodes):
        for node in nodes:
            if isinstance(node, LatexCharsNode):
                result.append(node.chars)
            elif isinstance(node, LatexMacroNode):
                # Optional: handle common formatting macros
                if node.macroname in {'em', 'textbf', 'textit', 'newblock'}:
                    _extract_text(node.nodeargd.argnlist)
                elif node.macroname == '~':
                    result.append(' ')
                else:
                    continue  # Skip unhandled macros
            elif isinstance(node, LatexEnvironmentNode):
                _extract_text(node.nodelist)
            elif hasattr(node, 'nodelist'):
                _extract_text(node.nodelist)

    _extract_text(nodelist)

    # Final cleanup
    text = ''.join(result)
    text = re.sub(r'\\end\{thebibliography\}.*', '', text, flags=re.DOTALL)
    text = re.sub(r'\\(input|newpage|end)\{[^}]*\}', '', text)
    text = re.sub(r'~', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
def parse_bibitem_block(tex_content):
    pattern = r'\\bibitem(?:\[[^\]]*\])?\{([^\}]+)\}(.*?)(?=(\\bibitem|\Z))'
    matches = re.findall(pattern, tex_content, re.DOTALL)
    parsed = []

    for key, body, _ in matches:
        clean_body = clean_latex_text(body)
        parsed.append({
            'key': key.strip(),
            'raw': clean_body
        })

    return parsed


In [10]:
def parse_latex_references(tex_path):
    with open(tex_path, 'r', encoding='utf-8') as f:
        tex_content = f.read()
    return parse_bibitem_block(tex_content)

In [12]:
from pprint import pformat

# --- Batch Processing Script ---

if __name__ == '__main__':
    folder_path = '/content/Latex-papers'  # Folder containing .tex files
    output_path = '/content/parsed_references.txt'

    with open(output_path, 'w', encoding='utf-8') as out_file:
        for filename in os.listdir(folder_path):
            if filename.endswith('.tex'):
                full_path = os.path.join(folder_path, filename)
                print(f"\n📄 Processing: {filename}")
                try:
                    references = parse_latex_references(full_path)
                    out_file.write(f"File: {filename}\n")
                    out_file.write(pformat(references, indent=2))
                    out_file.write("\n\n" + "="*80 + "\n\n")
                except Exception as e:
                    error_msg = f"❌ Failed to parse {filename}: {e}"
                    print(error_msg)
                    out_file.write(f"{error_msg}\n\n")



📄 Processing: 0704.0001.tex

📄 Processing: 1905.10003.tex

📄 Processing: 0704.0002.tex

📄 Processing: 1905.01664.tex

📄 Processing: 1905.01024.tex

📄 Processing: 1905.10000.tex

📄 Processing: 0704.0059.tex


I asked Gemini:



Show more

Conversation with Gemini
Are the following references the same style:



    'raw': 'authorD.Acosta etal. (collaborationCDF Collaboration), '

           'journalPhys. Rev. Lett. volume95, pages022003 (year2005).'},





    'raw': 'Aldous, D.J. (1985). Exchangeability and related topics. In cole '

           "d't de Probabilits de Saint-Flour XIII—1983, pages 1198. "

           'Springer.'},



    'raw': 'Berg, A.R., Jordn, T.: Algorithms for graph rigidity and scene '

           'analysis. In: Proc. 11th European Symposium on Algorithms (ESA '

           "'03), LNCS, vol. 2832, pp. 7889. (2003)"},





    'raw': "A. D. Aleksandrov and V. V. Strel'tsov, The isoperimetric problem "

           'and estimates of the length of a curve on a surface, Proc. Steklov '

           'Inst. Math. 76 (1965), 67-80; English transi., Proc. Steklov Inst '

           'Math. (1967), 81-99.'},



    'raw': 'Coffman, V., Kundu, J., Wootters, W.K.: Distributed entanglement. '

           'Phys. Rev. A 61, 052306 (2000)'},



    'raw': 'Bakos, G., Noyes, R.W., Kovcs, G., Stanek, K.Z., Sasselov, D.D., '

           'Domsa, I. 2004, , 116, 266'},


The references provided are **not** all in the same style.



* **Reference 1 (Acosta et al.):**
    * **Style indicators:** Author last name, initials, collaboration name, journal abbreviation, volume, pages, year in parentheses. This looks like a common style for physics journals (e.g., APS style).
    * `authorD.Acosta etal. (collaborationCDF Collaboration), journalPhys. Rev. Lett. volume95, pages022003 (year2005).`

* **Reference 2 (Aldous):**
    * **Style indicators:** Author last name, initials, year in parentheses after author, title of work, "In" followed by edited volume title, pages, publisher. This is often seen in conference proceedings or edited books, sometimes resembling a variant of APA or Chicago.
    * `Aldous, D.J. (1985). Exchangeability and related topics. In cole d't de Probabilits de Saint-Flour XIII—1983, pages 1198. Springer.`

* **Reference 3 (Berg, Jordán):**
    * **Style indicators:** Author last name, initials (sometimes separated by a colon), title, "In:" followed by conference name/proceedings, LNCS (Lecture Notes in Computer Science) volume, page range, year in parentheses. This is very common in computer science conference proceedings.
    * `Berg, A.R., Jordn, T.: Algorithms for graph rigidity and scene analysis. In: Proc. 11th European Symposium on Algorithms (ESA '03), LNCS, vol. 2832, pp. 7889. (2003)`

* **Reference 4 (Aleksandrov and Strel'tsov):**
    * **Style indicators:** Initials first, then last name for authors, title of work, journal name (abbreviated), volume, year, page range, and then a separate entry for English translation. This particular style with the "English transi." separated out is less common but indicates a specific format.
    * `A. D. Aleksandrov and V. V. Strel'tsov, The isoperimetric problem and estimates of the length of a curve on a surface, Proc. Steklov Inst. Math. 76 (1965), 67-80; English transi., Proc. Steklov Inst Math. (1967), 81-99.`

* **Reference 5 (Coffman, Kundu, Wootters):**
    * **Style indicators:** Author last name, initials (sometimes with no comma or space between last name and initial), title, journal abbreviation, volume, page number, year in parentheses. Similar to the first one, likely a physics journal style.
    * `Coffman, V., Kundu, J., Wootters, W.K.: Distributed entanglement. Phys. Rev. A 61, 052306 (2000)`

* **Reference 6 (Bakos et al.):**
    * **Style indicators:** Last name, initials, then year, then some numbers that might correspond to volume and page (though the formatting is unusual with double commas and no explicit journal title). This looks incomplete or highly idiosyncratic.
    * `Bakos, G., Noyes, R.W., Kovcs, G., Stanek, K.Z., Sasselov, D.D., Domsa, I. 2004, , 116, 266`

**In summary:** There are variations in author-year placement, punctuation, abbreviations, and the inclusion of elements like "In:" or "Proc." which clearly indicate different citation styles are being used across these examples.

Extract --> Yes

Where do we go from here?

Classify + Recognize --> Not sure